In [3]:
# 5-15
# 1
# 650, 1.1
# 1.1, 1.1
# 110, 2.2, 3.3, 165

In [4]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [5]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [6]:
dprice = 1

dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [7]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [8]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


# activation function

## relu

In [9]:
import numpy as np

In [33]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy() # x > 0
        out[self.mask] = 0 # x <= 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0 # x <= 0
        dx = dout # x > 0
        
        return dx

In [13]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x <- 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


## sigmoid

### $ sigmoid(x)' = \frac{e^{-x}}{(1 + e^{-x})^2} = \frac{(1 + e^{-x} - 1)}{(1 + e^{-x})^2} = y - y^2 = y(1 - y) $

In [14]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

## Affine

In [15]:
X_dot_W = np.array([[0,0,0],[10,10,10]])
B = np.array([1,2,3])

print (X_dot_W)

[[ 0  0  0]
 [10 10 10]]


In [16]:
print(X_dot_W + B)

array([[ 1,  2,  3],
       [11, 12, 13]])

In [18]:
dY = np.array([[1,2,3], [4,5,6]])
dB = np.sum(dY, axis=0)

print(dB)

[5 7 9]


In [19]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

## softmax with loss

### cross entropy error

## $ \frac{d(\sum_k-{t_klog(y_k)})}{dy} = \sum_k{-\frac{t_k}{y_k}} $

## $ \frac{d (crossentopy(softmax(x), t))}{dL} = \sum_k{y_k - t_k} $

In [20]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [21]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

# Neural Net

In [30]:
def softmax(x):
    C = np.max(x)
    e = np.exp(x - C)
    s = np.sum(e)
    
    return e / s

In [31]:
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = weight_init_std * np.random.randn(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = weight_init_std * np.random.randn(output_size)
        
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = np.argmax(t, axis=1)
            
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

# test

In [23]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784')

dat = mnist.data.to_numpy()
tar = mnist.target.to_numpy()

x_train = dat[:60000,]
x_test = dat[60000:,]
t_train = tar[:60000]
t_test = tar[60000:]

I = np.eye(10)
t_train = I[t_train.astype(int)]
t_test = I[t_test.astype(int)]

In [35]:
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = 500

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc : " + str(train_acc) + " | " + str(test_acc))

train acc, test acc : 0.09808333333333333 | 0.0976


C:\Users\nemok\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in subtract
  This is separate from the ipykernel package so we can avoid doing imports until


train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
train acc, test acc : 0.09871666666666666 | 0.098
